In [1]:
import numpy as np
import cv2 as cv
from motrackers import SimpleTracker2
from motrackers.utils import select_caffemodel, select_videofile

### Setup 

#### Choose file paths


In [2]:
video_file = select_videofile('..')
prototxt, weights = select_caffemodel('..')
display(video_file, prototxt, weights)

FileChooser(path='..', filename='', show_hidden='False')

FileChooser(path='..', filename='', show_hidden='False')

FileChooser(path='..', filename='', show_hidden='False')

#### Set hyper parameters and constants

In [4]:
print_tracks = True   # print the tracker output if True

video = video_file.selected

model = {"prototxt": prototxt.selected,
         "weights": weights.selected,
         "object_names": {0: 'background', 
                          1: 'aeroplane', 
                          2: 'bicycle', 
                          3: 'bird',
                          4: 'boat',
                          5: 'bottle',
                          6: 'bus', 
                          7: 'car', 
                          8: 'cat', 
                          9: 'chair',
                          10: 'cow', 
                          11: 'diningtable', 
                          12: 'dog', 
                          13: 'horse',
                          14: 'motorbike', 
                          15: 'person', 
                          16: 'pottedplant',
                          17: 'sheep', 
                          18: 'sofa', 
                          19: 'train',
                          20: 'tvmonitor'},
         "threshold": 0.2,
         "confidence_threshold": 0.2,
         "pixel_std":1/127.5,
         "pixel_mean": 127.5,
         "input_size": (300, 300)
        }

max_object_lost_count = 5   # maximum number of object losts counted when the object is being tracked

np.random.seed(12345)
bbox_colors = {key: np.random.randint(0, 255, size=(3,)).tolist() for key in model['object_names'].keys()}

### Initialize
* Initialize video
* Initialize network
* Initialize tracker

In [5]:
cap = cv.VideoCapture(video)
net = cv.dnn.readNetFromCaffe(model["prototxt"], model["weights"])
tracker = SimpleTracker2(max_lost=max_object_lost_count)

In [6]:
(H, W) = (None, None)
writer = None

while True:
    ok, image = cap.read()
    
    if not ok:
        print("Cannot read the video feed.")
        break
    
    if W is None or H is None: 
        (H, W) = image.shape[:2]
    
    image_resized = cv.resize(image, model["input_size"])

    blob = cv.dnn.blobFromImage(image_resized, 
                                 model["pixel_std"], 
                                 model["input_size"], 
                                 (model["pixel_mean"], model["pixel_mean"], model["pixel_mean"]), 
                                 False)

    net.setInput(blob)
    detections = net.forward()

    rows = image_resized.shape[0]
    cols = image_resized.shape[1]
    
    boxes, confidences, classIDs, detections_bbox = [], [], [], []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > model['confidence_threshold']:
            class_id = int(detections[0, 0, i, 1])

            # object location 
            left = int(detections[0, 0, i, 3] * cols) 
            top = int(detections[0, 0, i, 4] * rows)
            right = int(detections[0, 0, i, 5] * cols)
            bottom = int(detections[0, 0, i, 6] * rows)
            
            # scaling factor of image
            height_factor = image.shape[0]/float(model["input_size"][0])
            width_factor = image.shape[1]/float(model["input_size"][1])
            
            # scale object detection bounding box to original image
            left = int(width_factor * left) 
            top = int(height_factor * top)
            
            right = int(width_factor * right)
            bottom = int(height_factor * bottom)
            
            width = right - left
            height = bottom - top
            
            boxes.append([left, top, width, height])
            confidences.append(float(confidence))
            classIDs.append(int(class_id))
        
    indices = cv.dnn.NMSBoxes(boxes, confidences, model["confidence_threshold"], model["threshold"])
    
    class_ids_to_track = []
    if len(indices)>0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            
            detections_bbox.append((x, y, x+w, y+h))
            class_ids_to_track.append(classIDs[i])
            
            clr = [int(c) for c in bbox_colors[classIDs[i]]]
            cv.rectangle(image, (x, y), (x+w, y+h), clr, 2)
            
            label = "{}:{:.4f}".format(model["object_names"][classIDs[i]], confidences[i])
            
            (label_width, label_height), baseLine = cv.getTextSize(
                label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            
            y_label = max(y, label_height)
            
            cv.rectangle(image, 
                         (x, y_label-label_height),
                         (x+label_width, y_label+baseLine), 
                         (255, 255, 255), 
                         cv.FILLED)
            
            cv.putText(image, label, (x, y_label), cv.FONT_HERSHEY_SIMPLEX, 0.5, clr, 2)
        
    tracks = tracker.update(detections_bbox, class_ids_to_track)
    if print_tracks:
        print("Tracker Outputs:\n", tracks, "\n")
    
    for track in tracks:
        track_count, trackid, class_id, centroid, track_bbox, track_info = track
        
        text = "ID {}".format(trackid)
        
        cv.putText(image, 
                   text, 
                   (centroid[0] - 10, centroid[1] - 10),
                   cv.FONT_HERSHEY_SIMPLEX,
                   0.5, 
                   (0, 255, 0), 
                   2)
        
        cv.circle(image, 
                  (centroid[0], centroid[1]), 
                  4, 
                  (0, 255, 0), 
                  -1)
    
    cv.imshow("image", image)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
    if writer is None:
        fourcc = cv.VideoWriter_fourcc(*"MJPG")
        writer = cv.VideoWriter("output.avi", fourcc, 30, (W, H), True)
    writer.write(image)

writer.release()
cap.release()
cv.destroyWindow("image")

[(1, 0, 15, array([590, 299]), (541, 126, 640, 473), {'lost': 0}), (1, 1, 15, array([315, 246]), (285, 156, 345, 337), {'lost': 0}), (1, 2, 15, array([533, 194]), (486, 20, 580, 368), {'lost': 0}), (1, 3, 4, array([345,  45]), (217, 6, 473, 84), {'lost': 0})]
[(2, 0, 15, array([590, 299]), (541, 126, 640, 473), {'lost': 1}), (2, 1, 15, array([312, 248]), (281, 158, 343, 339), {'lost': 0}), (2, 2, 15, array([542, 224]), (492, 12, 593, 436), {'lost': 0}), (2, 3, 4, array([345,  45]), (217, 6, 473, 84), {'lost': 1})]
[(3, 0, 15, array([588, 284]), (539, 105, 637, 464), {'lost': 0}), (3, 1, 15, array([312, 248]), (281, 158, 343, 339), {'lost': 1}), (3, 2, 15, array([542, 224]), (492, 12, 593, 436), {'lost': 1}), (3, 3, 4, array([345,  45]), (217, 6, 473, 84), {'lost': 2})]
[(4, 0, 15, array([580, 245]), (526, 14, 635, 476), {'lost': 0}), (4, 1, 15, array([311, 246]), (279, 155, 343, 337), {'lost': 0}), (4, 2, 15, array([542, 224]), (492, 12, 593, 436), {'lost': 2}), (4, 3, 4, array([345,  

[(39, 0, 15, array([348, 242]), (266, 30, 430, 454), {'lost': 0}), (39, 4, 16, array([551, 154]), (499, 1, 603, 308), {'lost': 0}), (39, 5, 15, array([406, 174]), (377, 68, 435, 281), {'lost': 0})]
[(40, 0, 15, array([344, 240]), (262, 28, 426, 452), {'lost': 0}), (40, 4, 16, array([554, 151]), (494, 1, 614, 302), {'lost': 0}), (40, 5, 15, array([406, 174]), (377, 68, 435, 281), {'lost': 1})]
[(41, 0, 15, array([341, 235]), (262, 17, 420, 454), {'lost': 0}), (41, 4, 16, array([554, 157]), (488, 12, 620, 302), {'lost': 0}), (41, 5, 15, array([406, 174]), (377, 68, 435, 281), {'lost': 2})]
[(42, 0, 15, array([329, 233]), (256, 17, 403, 449), {'lost': 0}), (42, 4, 16, array([554, 157]), (488, 12, 620, 302), {'lost': 1}), (42, 5, 15, array([406, 174]), (377, 68, 435, 281), {'lost': 3})]
[(43, 0, 15, array([326, 232]), (251, 27, 401, 438), {'lost': 0}), (43, 4, 16, array([554, 157]), (488, 12, 620, 302), {'lost': 2}), (43, 5, 15, array([406, 174]), (377, 68, 435, 281), {'lost': 4})]
[(44, 0

[(76, 0, 15, array([159, 254]), (85, 86, 234, 422), {'lost': 0}), (76, 6, 16, array([ 41, 241]), (2, 196, 81, 286), {'lost': 1}), (76, 8, 15, array([516, 256]), (462, 104, 571, 409), {'lost': 0}), (76, 10, 4, array([344,  44]), (228, 6, 460, 83), {'lost': 4}), (76, 11, 15, array([329, 226]), (294, 136, 364, 316), {'lost': 0})]
[(77, 0, 15, array([148, 253]), (78, 84, 219, 422), {'lost': 0}), (77, 6, 16, array([ 41, 241]), (2, 196, 81, 286), {'lost': 2}), (77, 8, 15, array([509, 266]), (462, 107, 556, 425), {'lost': 0}), (77, 10, 4, array([344,  44]), (228, 6, 460, 83), {'lost': 5}), (77, 11, 15, array([328, 231]), (294, 139, 362, 324), {'lost': 0})]
[(78, 0, 15, array([146, 250]), (76, 84, 217, 417), {'lost': 0}), (78, 6, 16, array([ 41, 241]), (2, 196, 81, 286), {'lost': 3}), (78, 8, 15, array([506, 267]), (456, 110, 556, 425), {'lost': 0}), (78, 11, 15, array([329, 234]), (296, 142, 362, 326), {'lost': 0})]
[(79, 0, 15, array([145, 259]), (78, 89, 213, 430), {'lost': 0}), (79, 6, 16,

[(100, 0, 15, array([ 81, 247]), (14, 91, 149, 404), {'lost': 0}), (100, 8, 15, array([386, 251]), (339, 107, 433, 396), {'lost': 0}), (100, 11, 15, array([322, 225]), (290, 128, 354, 323), {'lost': 0}), (100, 15, 15, array([553, 243]), (492, 8, 614, 478), {'lost': 0})]
[(101, 0, 15, array([ 80, 243]), (10, 91, 151, 395), {'lost': 0}), (101, 8, 15, array([389, 252]), (343, 110, 435, 395), {'lost': 0}), (101, 11, 15, array([322, 225]), (290, 128, 354, 323), {'lost': 1}), (101, 15, 15, array([553, 244]), (490, 8, 616, 480), {'lost': 0})]
[(102, 0, 15, array([ 81, 254]), (8, 100, 155, 409), {'lost': 0}), (102, 8, 15, array([375, 250]), (317, 113, 433, 387), {'lost': 0}), (102, 11, 15, array([322, 225]), (290, 128, 354, 323), {'lost': 2}), (102, 15, 15, array([537, 233]), (477, 11, 597, 456), {'lost': 0})]
[(103, 0, 15, array([ 79, 251]), (4, 96, 155, 406), {'lost': 0}), (103, 8, 15, array([369, 248]), (320, 110, 418, 387), {'lost': 0}), (103, 11, 15, array([322, 225]), (290, 128, 354, 323

[(129, 8, 15, array([312, 254]), (224, 44, 401, 465), {'lost': 0}), (129, 15, 15, array([416, 258]), (349, 64, 484, 452), {'lost': 0}), (129, 19, 16, array([551, 172]), (494, 4, 608, 340), {'lost': 0}), (129, 20, 15, array([ 76, 236]), (32, 107, 121, 366), {'lost': 0})]
[(130, 8, 15, array([307, 250]), (221, 46, 394, 454), {'lost': 0}), (130, 15, 15, array([410, 276]), (337, 97, 484, 456), {'lost': 0}), (130, 19, 16, array([548, 164]), (492, 0, 605, 329), {'lost': 0}), (130, 20, 15, array([ 76, 236]), (32, 107, 121, 366), {'lost': 0}), (130, 21, 15, array([392, 156]), (356, 73, 428, 240), {'lost': 0}), (130, 22, 15, array([340,  69]), (307, 0, 373, 139), {'lost': 0})]
[(131, 8, 15, array([298, 249]), (215, 35, 381, 464), {'lost': 0}), (131, 15, 15, array([399, 279]), (326, 115, 473, 444), {'lost': 0}), (131, 19, 16, array([544, 168]), (490, 3, 599, 334), {'lost': 0}), (131, 20, 15, array([ 76, 236]), (32, 107, 121, 366), {'lost': 1}), (131, 21, 15, array([392, 156]), (356, 73, 428, 240

...

[(648, 54, 16, array([547, 164]), (486, 4, 608, 324), {'lost': 0}), (648, 57, 16, array([ 44, 267]), (4, 227, 85, 308), {'lost': 0}), (648, 58, 7, array([186, 273]), (85, 217, 288, 329), {'lost': 0}), (648, 59, 15, array([319, 237]), (283, 137, 356, 337), {'lost': 0})]
[(649, 54, 16, array([550, 164]), (492, 6, 608, 323), {'lost': 0}), (649, 57, 16, array([ 44, 267]), (4, 227, 85, 308), {'lost': 1}), (649, 58, 7, array([186, 273]), (87, 216, 285, 331), {'lost': 0}), (649, 59, 15, array([322, 242]), (285, 147, 360, 337), {'lost': 0})]
[(650, 54, 16, array([552, 164]), (492, 3, 612, 326), {'lost': 0}), (650, 57, 16, array([ 44, 265]), (4, 224, 85, 307), {'lost': 0}), (650, 58, 7, array([191, 274]), (89, 217, 294, 331), {'lost': 0}), (650, 59, 15, array([320, 243]), (283, 145, 358, 342), {'lost': 0}), (650, 60, 4, array([337,  43]), (221, 1, 454, 86), {'lost': 0})]
[(651, 54, 16, array([553, 166]), (492, 4, 614, 329), {'lost': 0}), (651, 57, 16, array([ 45, 265]), (4, 224, 87, 307), {'los

[(697, 54, 16, array([547, 158]), (494, 1, 601, 315), {'lost': 0}), (697, 63, 15, array([327, 218]), (294, 128, 360, 308), {'lost': 0}), (697, 64, 16, array([ 46, 239]), (4, 198, 89, 280), {'lost': 0})]
[(698, 54, 16, array([550, 156]), (488, -1, 612, 313), {'lost': 0}), (698, 63, 15, array([329, 216]), (296, 123, 362, 310), {'lost': 0}), (698, 64, 16, array([ 46, 239]), (4, 198, 89, 280), {'lost': 1})]
[(699, 54, 16, array([551, 156]), (488, 0, 614, 313), {'lost': 0}), (699, 63, 15, array([326, 215]), (292, 123, 360, 307), {'lost': 0}), (699, 64, 16, array([ 43, 238]), (2, 196, 85, 280), {'lost': 0})]
[(700, 54, 16, array([553, 157]), (490, 1, 616, 313), {'lost': 0}), (700, 63, 15, array([326, 215]), (292, 123, 360, 307), {'lost': 0}), (700, 64, 16, array([ 43, 239]), (2, 198, 85, 280), {'lost': 0})]
[(701, 54, 16, array([555, 156]), (494, -1, 616, 313), {'lost': 0}), (701, 63, 15, array([326, 215]), (292, 123, 360, 307), {'lost': 0}), (701, 64, 16, array([ 43, 239]), (2, 198, 85, 280

[(746, 54, 16, array([546, 155]), (490, 1, 603, 310), {'lost': 0}), (746, 63, 15, array([323, 211]), (288, 124, 358, 299), {'lost': 0})]
[(747, 54, 16, array([546, 157]), (490, 3, 603, 312), {'lost': 0}), (747, 63, 15, array([321, 213]), (285, 126, 358, 300), {'lost': 0})]
[(748, 54, 16, array([546, 156]), (492, 3, 601, 310), {'lost': 0}), (748, 63, 15, array([321, 210]), (285, 124, 358, 297), {'lost': 0})]
[(749, 54, 16, array([547, 159]), (494, 3, 601, 316), {'lost': 0}), (749, 63, 15, array([324, 210]), (290, 120, 358, 300), {'lost': 0})]
[(750, 54, 16, array([548, 161]), (494, 3, 603, 320), {'lost': 0}), (750, 63, 15, array([320, 211]), (285, 121, 356, 302), {'lost': 0})]
[(751, 54, 16, array([548, 161]), (494, 3, 603, 320), {'lost': 1}), (751, 63, 15, array([323, 212]), (290, 120, 356, 305), {'lost': 0})]
[(752, 54, 16, array([550, 156]), (497, 0, 603, 312), {'lost': 0}), (752, 63, 15, array([323, 212]), (290, 120, 356, 305), {'lost': 1})]
[(753, 54, 16, array([550, 155]), (497, 0

[(795, 63, 15, array([347, 202]), (311, 116, 384, 289), {'lost': 5}), (795, 65, 15, array([171, 240]), (0, 0, 343, 480), {'lost': 0}), (795, 68, 15, array([299, 132]), (219, 0, 379, 265), {'lost': 0}), (795, 69, 7, array([417, 256]), (369, 216, 465, 296), {'lost': 1})]
[(796, 63, 15, array([309, 240]), (196, 11, 422, 470), {'lost': 0}), (796, 65, 15, array([171, 240]), (0, 0, 343, 480), {'lost': 1}), (796, 68, 15, array([299, 132]), (219, 0, 379, 265), {'lost': 1}), (796, 69, 7, array([417, 256]), (369, 216, 465, 296), {'lost': 2})]
[(797, 63, 15, array([309, 240]), (196, 11, 422, 470), {'lost': 0}), (797, 65, 15, array([171, 240]), (0, 0, 343, 480), {'lost': 2}), (797, 68, 15, array([299, 132]), (219, 0, 379, 265), {'lost': 2}), (797, 69, 7, array([417, 256]), (369, 216, 465, 296), {'lost': 3})]
[(798, 63, 15, array([306, 233]), (189, -4, 424, 470), {'lost': 0}), (798, 65, 15, array([171, 240]), (0, 0, 343, 480), {'lost': 3}), (798, 68, 15, array([299, 132]), (219, 0, 379, 265), {'los

[(848, 73, 15, array([326, 192]), (288, 100, 364, 284), {'lost': 0}), (848, 74, 16, array([ 49, 219]), (8, 179, 91, 260), {'lost': 1})]
[(849, 73, 15, array([344, 197]), (307, 115, 381, 280), {'lost': 0}), (849, 74, 16, array([ 49, 219]), (8, 179, 91, 260), {'lost': 2})]
[(850, 73, 15, array([344, 197]), (307, 115, 381, 280), {'lost': 1}), (850, 74, 16, array([ 46, 218]), (6, 176, 87, 260), {'lost': 0})]
[(851, 73, 15, array([324, 193]), (290, 100, 358, 286), {'lost': 0}), (851, 74, 16, array([ 46, 218]), (6, 177, 87, 259), {'lost': 0})]
[(852, 73, 15, array([325, 195]), (290, 102, 360, 289), {'lost': 0}), (852, 74, 16, array([ 47, 218]), (8, 176, 87, 260), {'lost': 0})]
[(853, 73, 15, array([327, 195]), (294, 100, 360, 291), {'lost': 0}), (853, 74, 16, array([ 41, 220]), (2, 177, 81, 264), {'lost': 0})]
[(854, 73, 15, array([326, 199]), (292, 102, 360, 296), {'lost': 0}), (854, 74, 16, array([ 41, 219]), (2, 177, 81, 262), {'lost': 0})]
[(855, 73, 15, array([327, 198]), (290, 99, 364,